In [ ]:
from bending_validation import *
import elastic_rods, sparse_matrices, pickle, scipy, linkage_vis, numpy as np, time

We can now save and restore full rod state to debug bad configurations:

In [ ]:
r = pickle.load(open('bad_rod.pkl', 'rb'))
m = elastic_rods.RodMaterial()
m.setEllipse(200, 0.3, 0.01, 0.005)
r.setMaterial(m)
fixedVars = [0, 1, 2, 77, 150, 151, 152]
view = linkage_vis.LinkageViewer(r, width=600, height=400)
view.setCameraParams([[ 0.8559,  3.2282,  0.8117],
       [-0.1116,  0.1192,  0.9865],
       [ 0.0694, -0.1334, -0.3078]])
view.show()

The twist discontinuity was due to a bug in the frame vectors to frame rotation conversion, which is now fixed.

Also, the new equilibrium solver quickly escapes the negatively curved region:

In [ ]:
elastic_rods.compute_equilibrium_knitro(r, niter=10, fixedVars=fixedVars)
view.update()

In [ ]:
elastic_rods.compute_equilibrium(r, verbose=True, fixedVars=fixedVars, niter=1, useIdentityMetric = False)
view.update()

In [ ]:
pickle.dump(r, open('rod_indefinite_demo.pkl', 'wb'))

In [ ]:
H = r.hessian()
#H.reflectUpperTriangle()
H.dumpBinary('H_badrod.mat')

The current challenge is to escape saddle points (where the gradient is exactly zero)

In [ ]:
L, a, n = 1, 0.1, 51
perturb = False
r, fixedVars = bendingTestRod(L, a, n)
pts, thetas = initialConfiguration(L, a, n, perturb)
thetas = np.ones(n - 1) * (np.pi / 2)
r.setDeformedConfiguration(pts, thetas)

view = linkage_vis.LinkageViewer(r, width=1024, height=600)
view.setCameraParams(((0.8471149305506829, 2.303483674764291, 0.4579086561101697),
 (-0.05274977528957473, 0.1226800644593855, 0.9910434213449677),
 (-0.41643357695558714, 0.21249216220933723, -0.36668979830898635)))
view.show()

In [ ]:
elastic_rods.compute_equilibrium_knitro(r, verbose=True, fixedVars=fixedVars, niter=10)
view.update()

In [ ]:
elastic_rods.compute_equilibrium(r, verbose=True, fixedVars=fixedVars, niter=10, gradTol=1e-6)
view.update()

In [ ]:
from compute_vibrational_modes import compute_vibrational_modes, MassMatrixType
import mode_viewer

lambdas, modes = compute_vibrational_modes(r, fixedVars, n=4)
mview = mode_viewer.ModeViewer(r, modes, lambdas)
mview.show()

Rest length solve: minimize elastic energy with respect to both the rest lengths and the deformed configuration.
This works as long as the bending is below some critical threshold, as seen below (compare a = 0.44 vs a = 0.47):

In [ ]:
L, a, n = 1, 0.47, 51
r, fixedVars = bendingTestRod(L, a, n)
pts, thetas = initialConfiguration(L, a, n, False)
r.setDeformedConfiguration(pts, thetas)

with suppress_stdout():
    elastic_rods.compute_equilibrium(r, verbose=True, fixedVars=fixedVars, niter=100)

rlview = linkage_vis.LinkageViewer(r, width=800, height=700)
rlview.setCameraParams(((-0.08773027702370209, 0.13519066571762273, -4.1987952798224795),
 (-0.04840963028921097, 0.958400758921313, -0.28129076236931344),
 (0.01975715401252783, -0.36464795573335124, 0.12497624338029313)))
rlview.show()

In [ ]:
with suppress_stdout():
    elastic_rods.restlen_solve(r, verbose=True, fixedVars=fixedVars, niter=5)
rlview.update()